### Замерим качество Линейной регрессии после обработки данных не просто на отложенной выборке, но и на Кросс-Валидации на 4 фолдах!

In [1]:
import numpy as np
import pandas as pd

processed_data = pd.read_csv('/Users/alikhansainov/Desktop/Karpov/processed_data copy.csv', index_col='id')

In [2]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


#### ! Не перемешивайте данные

In [3]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.linear_model import LinearRegression
selector = KFold(n_splits=4)
import numpy as np




initial_cols = ['vendor_id', 'passenger_count', 'store_and_fwd_flag','distance_km','log_trip_duration']

initial_data = processed_data[initial_cols]

from sklearn.model_selection import train_test_split
X = initial_data.drop('log_trip_duration', axis=1)
y = initial_data['log_trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)


In [4]:
### Шаг №5
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный selector

### Your code is here

### Выделим test
losses_test = []
losses_train = []
splitter = KFold(n_splits=5, shuffle=True, random_state=42)
for train_fold, test_fold in splitter.split(X_train):
    Xtrain, Xtest = X_train.values[train_fold], X_train.values[test_fold]
    ytrain, ytest = y_train.values[train_fold], y_train.values[test_fold]
    model = LinearRegression()
    model.fit(Xtrain, ytrain)
    losses_train.append(np.mean ((model.predict(Xtrain) - ytrain) ** 2))
    losses_test.append(np.mean ((model.predict(Xtest) - ytest) ** 2))
cross_val_error_train = np.mean(losses_test)
cross_val_error_test = np.mean(losses_train)
print ("Train error: ",(cross_val_error_train))
print ("Train error: ",(cross_val_error_test))
print(f"MSLE на Кросс-валидации: {round(cross_val_error_train, 3)}")
print(f"MSLE на Кросс-валидации: {round(cross_val_error_test, 3)}")

Train error:  0.42999494567572843
Train error:  0.4286164081584971
MSLE на Кросс-валидации: 0.43
MSLE на Кросс-валидации: 0.429


## Поработал один из хитрых гномов!

В отличие от своих собратьев, третий гном оказался тем еще бездельником в школьные годы, но все равно страстно желал во всем догнать первых двух. И сейчас, желая помочь им в построении модели по предсказанию длительности поездки такси, добавил в данные 20 зашифрованных фичей (их смысл нам не рассказали: какая-то секретная информация о водителях).

Гном думал следующим образом: "Ну не может же модель стать хуже! А тут вот авось и мое нововведение уменьшит ошибку в разы! Тогда и меня станут звать на гномий  data-саммит."

Проверим на кросс-валидации, насколько гном оказался прав!

In [5]:
new_data = pd.read_csv('/Users/alikhansainov/Desktop/Karpov/new_data.csv', index_col='id')

In [6]:
new_data.head()


,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,1,1,1,1,1,...,0,0,0,0,0,1.500479,2.251437,3.378234,5.068969,7.605881
id2377394,0,930.399753,0,1.807119,6.498282,0,0,0,0,0,...,0,0,0,0,0,1.807119,3.265681,5.901475,10.664670,19.272331
id3858529,1,930.399753,0,6.392080,7.661527,1,1,1,1,1,...,0,0,0,0,0,6.392080,40.858690,261.172025,1669.432545,10671.146803
id3504673,1,930.399753,0,1.487155,6.063785,1,1,1,1,1,...,0,0,0,0,0,1.487155,2.211629,3.289035,4.891303,7.274125
id2181028,1,930.399753,0,1.189925,6.077642,1,1,1,1,1,...,0,0,0,0,0,1.189925,1.415923,1.684842,2.004837,2.385606


In [7]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
import numpy as np

new_data = pd.read_csv('/Users/alikhansainov/Desktop/Karpov/new_data.csv', index_col='id')

selector = KFold(n_splits=4)
X2 = new_data.drop('log_trip_duration', axis=1)
y2 = new_data['log_trip_duration']

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

model_2 = LinearRegression()
model_2.fit(X_train2, y_train2)

cross_val_error_2 = cross_validate(model_2, X2, y2, scoring='neg_mean_squared_error', cv=selector, return_train_score=True)

print(round(np.mean(-cross_val_error_2['test_score']), 3))


140.921


В линейной алгербре зачастую используют понятие **ранга матрицы**. Оно соответствует кол-ву линейно независимых столбцов в матрице. Иными словами, позволяет оценить, есть ли избыток информации в нашем датафрейме. Если ранг матрицы меньше, чем кол-во используемых столбцов, то некоторые фичи следует удалить, ведь иначе возникает ситуация строгой мультиколлинеарности.

Чтобы замерить ранг в наших матрицах объект-признак, можно воспользоваться функцией numpy.linalg.matrix_rank

Константным признаком в данном упражнении можно пренебречь.

In [8]:
import numpy as np

# Ранг изначальной матрицы с данными
rank_processed = np.linalg.matrix_rank(initial_data.values)

# Ранг матрицы третьего гнома
rank_new = np.linalg.matrix_rank(new_data.values)

print("Ранг изначальной матрицы с данными:", rank_processed)
print("Ранг матрицы третьего гнома:", rank_new)



Ранг изначальной матрицы с данными: 5
Ранг матрицы третьего гнома: 5


In [9]:
### Создайте переменные num_features_processed, num_features_new
### Соответственно равные кол-ву фичей в изначальной матрицы
### с данными и кол-ву фичей у третьего гнома

### Your code is here




num_features_processed = X.shape[1]
num_features_new = X2.shape[1]
print (num_features_processed)
print (num_features_new)



4
24


In [10]:
### Шаг №7
print(f"В первой модели всего фичей: {num_features_processed}, - а ранг равен {rank_processed}")

print(f"Во второй модели всего фичей: {num_features_new}, - а ранг равен {rank_new}")

В первой модели всего фичей: 4, - а ранг равен 5
Во второй модели всего фичей: 24, - а ранг равен 5


Не кажется ли нам, что из-за новых 20 фичей появилась проблема мультиколлинеарности? Как поступить гному, чтобы, с одной стороны, получить адекватное качество, а с другой стороны, не повредить свое самолюбие и не убирать новые признаки?

Верно! Например, с помощью регуляризации.

Найдите такой параметр регуляризации $\alpha$ для Ridge и Lasso случая, чтобы ошибка MSLE на кросс-валидации оказалась строго меньше 0.4

**ALARM**: используйте процедуру масштабирования данных (воспользуйтесь методом MinMaxScaler) перед тем как применить регуляризацию. Важно - чтобы сохранить концепцию независимости обучения на трейне и на тесте, на каждой итерации кросс-валидации необходимо замерять параметры стандартизации исключительно на трейне, а потом применять на валидационном фолде.

In [11]:
### Пример, как это можно сделать в цикле
### То есть обучить Lasso, учитывая масштабирование
### Исключительно на трейне на каждой итерации



X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso

scores = []

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    ### Фитим исключительно на трейне!
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    ### max_iter иногда требуется ставить побольше, 
    ### особенно когда данных много и/или они сложные
    ### этот параметр регулирует верхнюю границу кол-ва
    ### итераций во время обучения
    ### подробнее - в документации
    
    ### По дефолту здесь параметр регуляризации alpha=1
    
    model_lasso = Lasso(max_iter=100000) 
    model_lasso.fit(X_train_scaled, Y_train)
    
    predictions = model_lasso.predict(X_test_scaled)
    
    scores.append(np.mean((predictions - Y_test)**2))

    
print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")

### P.S. если вы уже умеете работать с конструкциями
### Pipeline, GridSearchCV, cross_validate
### Можете использовать их. Мы познакомимся с ними позже.

MSLE на Кросс-валидации равен: 0.6332330617999488


In [12]:
### Пример, как это можно сделать в цикле
### То есть обучить Lasso, учитывая масштабирование
### Исключительно на трейне на каждой итерации

X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
kf = KFold(n_splits=5, shuffle=True, random_state=42)
alphas = np.logspace(-3, 3, 7)
best_msle = 0.4
for alpha in alphas:
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.values[train_index], X.values[test_index]
        Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    ### Фитим исключительно на трейне!
        scaler = MinMaxScaler()
        scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
    
    ### max_iter иногда требуется ставить побольше, 
    ### особенно когда данных много и/или они сложные
    ### этот параметр регулирует верхнюю границу кол-ва
    ### итераций во время обучения
    ### подробнее - в документации
    
    ### По дефолту здесь параметр регуляризации alpha=1
        model_ridge = Ridge(alpha = alpha) 
        model_ridge.fit(X_train_scaled, Y_train)
        predictions = model_ridge.predict(X_test_scaled)
        scores.append(np.mean((predictions - Y_test)**2))
        mean_msle = np.mean(scores)
        if mean_msle < best_msle:
            best_msle = mean_msle
            best_alpha = alpha
            print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")
            print (best_alpha)
        break
    break
### P.S. если вы уже умеете работать с конструкциями
### Pipeline, GridSearchCV, cross_validate
### Можете использовать их. Мы познакомимся с ними позже.


MSLE на Кросс-валидации равен: 0.37778622997687794
0.001


In [13]:
### Найдите оптимальный параметр регуляризации
### для случая Lasso
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Шаг №9
### Your code is here

X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
best_msle = 0.4
selector = KFold(n_splits=4)
alphas = [0.5,1.0,1.5]
for alpha in alphas:
    scores2 = []
    for train_index, test_index in selector.split(X):
        X_train, X_test = X.values[train_index], X.values[test_index]
        Y_train, Y_test = Y.values[train_index], Y.values[test_index]
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        model_lasso = Lasso(alpha=alpha, max_iter = 100000) 
        model_lasso.fit(X_train_scaled, Y_train)
        predictions = model_lasso.predict(X_test_scaled)
        scores2.append(np.mean((predictions - Y_test)**2))
        mean_msle = np.mean(scores2)
        if mean_msle < best_msle:
            best_msle = mean_msle 
            best_alpha = alpha
    if best_alpha is not None:
        print(f"MSLE на Кросс-валидации равен: {np.mean(scores2)}")
        print (best_alpha)
    else:
        print ("nothing found")


MSLE на Кросс-валидации равен: 0.6332330617999488
0.001
MSLE на Кросс-валидации равен: 0.6332330617999488
0.001
MSLE на Кросс-валидации равен: 0.6332330617999488
0.001


In [14]:
arr = np.zeros((4,4))
arr

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [15]:

from sklearn.linear_model import Lasso
model = Lasso (alpha = 15)
model.fit(X, y)



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.553e+05, tolerance: 9.237e+01
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=15)